In [4]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath(os.path.dirname("__file__")), '..'))
from utils import data_handling, model_development
import matplotlib.pyplot as plt
from ipywidgets import interact

In [5]:
##
# Configure the model training policy:
#
model_development.configure_training_policy()

In [6]:
##
# Load and preprocess data:
#
cell_images, cell_labels, cell_types = data_handling.load_data_from_file('../data/labels.csv', True)
X, Y, Z = data_handling.preprocess_data(cell_images, cell_labels, cell_types)
X_train, Y_train, Z_train = X[: 2000], Y[: 2000], Z[: 2000]
X_val, Y_val, Z_val = X[2000 : ], Y[2000 : ], Z[2000 : ]

print([cell_images.shape, len(cell_labels), len(cell_types)])
print([X.shape, len(Y), len(Z)])

[(2624, 300, 300), 2624, 2624]
[(2624, 300, 300, 3), 2624, 2624]


In [7]:
@interact(n = (0, cell_images.shape[0] - 1))
def display_data(n = 0):
    fig, axs = plt.subplots()
    axs.imshow(cell_images[n], cmap = 'gray')
    axs.set_title(f'Class: {cell_labels[n]} - Type: {cell_types[n]}')
    return None

interactive(children=(IntSlider(value=0, description='n', max=2623), Output()), _dom_classes=('widget-interact…

In [8]:
@interact(n = (0, X.shape[0] - 1))
def display_data(n = 0):
    fig, axs = plt.subplots()
    axs.imshow(X[n], cmap = 'gray')
    axs.set_title(f'Class: {Y[n]} - Type: {Z[n]}')
    return None

interactive(children=(IntSlider(value=0, description='n', max=2623), Output()), _dom_classes=('widget-interact…